In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('max_columns', None)

In [ ]:
def getDataframeFromNethogsOutput(df):
    df['timestamp'] = pd.to_datetime(df[0].str.strip('['))
    df[['sent','received']] = df[1].str.split(' ', expand=True)[[2,3]].astype(float)
    df = df.drop([0,1],axis=1)
    return df

In [ ]:
def getAverageByMinute(df):
    df1 = df.copy().drop(['timestamp'],axis=1)
    # convert to datetime
    df1['datetime'] = pd.to_datetime(df['timestamp'])
    df1 = df1.set_index('datetime')
    # get average in every minute
    df1 = df1.resample('1T').mean()
    df1 = df1.reset_index()
    df1 = df1.rename(columns={"datetime": "timestamp"})
    return df1

In [ ]:
df1 = pd.read_csv('1205_nethogs_ipfs.txt', sep=']', header=None)

df1 = getDataframeFromNethogsOutput(df1)
df1 = getAverageByMinute(df1)
df1['node'] = 'ipfs'

df1.shape

In [ ]:
df1.describe()

In [ ]:
df2 = pd.read_csv('1205_nethogs_swarm.txt', sep=']', header=None)

df1 = getDataframeFromNethogsOutput(df2)
df2 = getAverageByMinute(df2)
df2['node'] = 'swarm'

df2.shape

In [ ]:
df2.describe()

In [ ]:
df = pd.concat([df1, df2])
df = df.fillna(0)
df.head()

In [ ]:
fig = px.line(df, x='timestamp', y='sent', color='node')
fig.update_yaxes(title="upload bandwidth (KB/second)")
fig.show()

In [ ]:
fig = px.line(df, x='timestamp', y='received', color='node')
fig.update_yaxes(title="download bandwidth (KB/second)")
fig.show()